In [ ]:
CURR_DIR = ".."

## Imports

In [ ]:
REQUIREMENTS_PATH = f"{CURR_DIR}/requirements/base.txt"

!pip install -r {REQUIREMENTS_PATH}

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys; sys.path.append(CURR_DIR)
import os
from src.main import run_ga
from src.utils.file_utils import analyse_results

## Experiments

### `berlin52`

In [ ]:
run_ga("berlin52", curr_dir=CURR_DIR)

In [ ]:
analyse_results(os.path.join(CURR_DIR, "data/results/berlin52"), "berlin52")

### `kroA100`

In [ ]:
run_ga("kroA100", curr_dir=CURR_DIR)

In [ ]:
analyse_results(os.path.join(CURR_DIR, "data/results/kroA100"), "kroA100")

### `pr1002`

In [ ]:
run_ga("pr1002", curr_dir=CURR_DIR, elitism_rate=0.12, tournament_size=15)

In [ ]:
analyse_results(os.path.join(CURR_DIR, "data/results/pr1002"), "pr1002", skip=5)